# RAG 流程
RAG的逻辑是先将本地库向量化，生成一个向量库，导入到大语言模型中，再将问题向量化，与向量库中的数据对比，输出答案。

需要用到：
- Embedding模型
- 向量数据库
- 大模型

使用 LangChain 库来搭建，参考文章：https://zhuanlan.zhihu.com/p/668082024

## 简要介绍 LangChain 库：
介绍来自于：https://github.com/liaokongVFX/LangChain-Chinese-Getting-Started-Guide  

LangChain是一个软件开发框架，可以更轻松地使用大型语言模型（LLM）创建应用程序。它是一个具有 Python 和 JavaScript 代码库的开源工具。LangChain 允许开发人员将 GPT-4 等 LLM 与外部数据相结合，为聊天机器人、代码理解、摘要等各种应用程序开辟了可能性。

LangChain 有几个组件（https://python.langchain.com/docs/integrations/components/）：
- Document loaders 对索引的支持
  - 加载数据, 拥有大量的文档加载器，比如 Email、Markdown、PDF、Youtube ... `langchain.document_loaders`用于导入.txt文件
  - 文档分割器 `langchain.text_splitter.CharacterTextSplitter`
- Embedding models
  - 向量化 `langchain.embeddings.HuggingFaceBgeEmbeddings`
- Vector stores
  - 向量库，对接向量存储与搜索，比如 Chroma、Pinecone、Qdrand `langchain.vectorstores.Chroma`
- Chat models 大语言模型


# 本地数据加载
使用 Loader 加载器，可以加载PDF，web，CSV，directory，HTIL，JSON等文件类型，官方：https://python.langchain.com/docs/how_to/#document-loaders  

当使用loader加载器读取到数据源后，数据源需要转换成 Document 对象后，后续才能进行使用

In [ ]:
from langchain.document_loaders import TextLoader  

loader = TextLoader('./data/jd.txt')
document = loader.load()

# Text Spltters 文本分割
为什么需要分割文本？因为我们每次不管是做把文本当作 prompt 发给 openai api ，还是还是使用 openai api embedding 功能都是有字符限制的。比如我们将一份300页的 pdf 发给 openai api，让他进行总结，他肯定会报超过最大 Token 错。所以这里就需要使用文本分割器去分割我们 loader 进来的 Document。

借助langchain的字符分割器, 假设采用固定字符长度分割chunk_size=128

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# 创建拆分器
"""
参数介绍
- separators - 分隔符字符串数组
- chunk_size - 每个文档的字符数量限制
- chunk_overlap - 两份文档重叠区域的长度
- length_function - 长度计算函数
- is_separator_regex - 如果为真：应当被解释为正则表达式，因此不需要转义。如果为假：应当被当作普通字符串分隔符，并转义任何特殊字符。
"""
text_spliter = CharacterTextSplitter(
    separator='\n\n',
    chunk_size = 128, 
    chunk_overlap = 20,
    length_function = len,
)

# 拆分文档，文档被拆分成n份，例如 {metadata 1}{metadata 2}{metadata 3}
documents = text_spliter.split_documents(document)

# 向量化&数据入库
这一步是要建立向量库，因为数据相关性搜索其实是向量运算。所以，不管我们是使用 openai api embedding 功能还是直接通过向量数据库直接查询，都需要将我们的加载进来的数据 Document 进行向量化，才能进行向量运算搜索。转换成向量也很简单，只需要我们把数据存储到对应的向量数据库中即可完成向量的转换。  

这里选用 m3e-base作为embedding模型，用 Chroma 建立向量库

官方Embedding模型：https://python.langchain.com/docs/integrations/text_embedding/
官方向量库：https://python.langchain.com/docs/integrations/vectorstores/

In [ ]:
# 第一次导入会自动下载，可能会有网络问题，在开头添加以下两句
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

# 向量化&数据入库
embedding = HuggingFaceBgeEmbeddings(
    model_name = "moka-ai/m3e-base",
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings': True},
    query_instruction="为文本生成向量表示用于文本检索",
)

# load data to Chroma 向量数据库
db = Chroma.from_documents(documents, embedding)

# 查询


In [ ]:
db.similarity_search('这是什么岗位？')
# 输出文档按照相关性大小排序 n 份结果，例如{metadata 1}{metadata 3}{metadata 2}
# 此时没有导入大语言模型，不会得到归纳后的答案